In [1]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

In [3]:
def scroll_page(browser):
    """
    Automatically control to scroll the webpage to the bottom
    Input: 
        browser: web driver
    Output: None
    """
    
    SCROLL_PAUSE_TIME = 2

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return None 

In [4]:
def get_product_links(source):
    """
    Retrive product links from a web source
    Input:
        webpage source
    Output:
        product links as list
    """
    soup = bs4.BeautifulSoup(source)
    domain = "https://products.wholefoodsmarket.com"
    contents = soup.find_all(name="a", class_="ProductCard-Root--3g5WI")
    links = [domain + content.get("href") for content in contents]
    return links

In [ ]:
product_links = []
browser = webdriver.Chrome()
init_url = "https://products.wholefoodsmarket.com/search?sort=relevance"
browser.get(init_url)

no_sub_categ = ["floral", "cheese", "seafood"]
buttons = browser.find_elements_by_class_name("LandingPage-SeeAll--2PvpU")
button_nums = len(buttons)

init_action = ActionChains(browser)
for i in range(0, button_nums):
    init_url = "https://products.wholefoodsmarket.com/search?sort=relevance"
    browser.get(init_url)

    buttons = browser.find_elements_by_class_name("LandingPage-SeeAll--2PvpU")
    button = buttons[i]
    init_action.click(button).perform()

    categ_url = browser.current_url
    browser.get(categ_url)
    print(categ_url)
    print(categ_url.split("=")[2])
    if categ_url.split("=")[2] not in no_sub_categ:
        sub_buttons = browser.find_elements_by_class_name("Radio-Label--1a5oe")
        sub_action = ActionChains(browser)
        for sub_button in sub_buttons[1:]:
            sub_action.click(sub_button).perform()
            scroll_page(browser)
            source = browser.page_source
            links = get_product_links(source)
            print("Number of products", len(links))
            product_links.extend(links)
            sub_action = ActionChains(browser)
    else:
        scroll_page(browser)
        source = browser.page_source
        links = get_product_links(source)
        print("Number of products", len(links))
        product_links.extend(links)

    back_action = ActionChains(browser)
    back_button = browser.find_element_by_class_name("Radio-Label--1a5oe")
    back_action.click(back_button).perform()
    print(browser.current_url)

    init_action = ActionChains(browser)
browser.close()

In [ ]:
browser.close()

In [ ]:
print(len(product_links))

In [ ]:
# 查找重复项
import collections
print([item for item, count in collections.Counter(product_links).items() if count > 1])

In [ ]:
print(product_links[545])

Find you may also like

In [ ]:
starting_link = product_links[545]
r = requests.get(starting_link)

In [ ]:
def get_what_you_may_like(starting_link):
    browser = webdriver.Chrome()
    browser.get(starting_link)
    source = browser.page_source
    browser.close()
    soup = bs4.BeautifulSoup(source)
    soups = soup.find_all("a", class_="ProductCard-Root--3g5WI")
    domain = "https://products.wholefoodsmarket.com"
    links = [domain + soup.get("href") for soup in soups]
    return links

In [ ]:
starting_link = "https://products.wholefoodsmarket.com/product/califia-farms-super-berry-probiotic-yogurt-drink-6e4114"
browser = webdriver.Chrome()
browser.get(starting_link)
source = browser.page_source
soup = bs4.BeautifulSoup(source)
soups = soup.find_all("a", class_="ProductCard-Root--3g5WI")

In [ ]:
for soup in soups:
    print(soup.get("href"))